In [ ]:
import os
import numpy as np 
import pandas as pd 

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
sample = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/sample_submission.csv")
valid = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv")
test_score = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv")
train_class = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv")
test = pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv")


In [ ]:
# cat_mtpl = {'obscene': 0.16, 'toxic': 0.32, 'threat': 1.5, 
#             'insult': 0.64, 'severe_toxic': 1.5, 'identity_hate': 1.5}

# for category in cat_mtpl:
#     train_class[category] = train_class[category] * cat_mtpl[category]

# train_class['score'] = train_class.loc[:, 'toxic':'identity_hate'].sum(axis=1)

# # train_class['score'] = train_class[['toxic', 'severe_toxic','obscene','threat','insult','identity_hate']].sum(axis = 1)
# train_class['y'] = np.where(train_class.score > 0, 1, 0)
# train_class

In [ ]:
# train_class.y.value_counts()

In [ ]:
# from scipy.stats import rankdata
# rankdata(train_class.score, method = 'dense')

In [ ]:
train_class

In [ ]:
# train_class.score.describe()

In [ ]:
train_class['y'] = train_class.loc[:,'toxic':'identity_hate'].sum(axis=1)

In [ ]:
# since the data is imbalanced, undersampling the majority class
# min_len = (train_class['y'] > 0).sum() 
# score_clean = train_class[train_class['y'] == 0].sample(n=min_len, random_state=123)  # take non toxic comments
# train_class = pd.concat([train_class[train_class['y'] > 0], score_clean])  # make new df
train_class.y.value_counts()

In [ ]:
X_train = train_class["comment_text"].str.lower()
X_test_less = valid["less_toxic"].str.lower()
X_test_more = valid["more_toxic"].str.lower()
test = test['text'].str.lower()
y_train = train_class[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values


In [ ]:
valid

In [ ]:
from keras.preprocessing import text, sequence


In [ ]:
max_features=100000
maxlen=150
embed_size=300

tok=text.Tokenizer(num_words=max_features,lower=True)
tok.fit_on_texts(list(X_train)+list(X_test_less)+list(X_test_more))
X_train=tok.texts_to_sequences(X_train)
X_test_less=tok.texts_to_sequences(X_test_less)
X_test_more=tok.texts_to_sequences(X_test_more)
test = tok.texts_to_sequences(test)

x_train=sequence.pad_sequences(X_train,maxlen=maxlen,padding='post')
X_test_less=sequence.pad_sequences(X_test_less,maxlen=maxlen, padding='post')
X_test_more=sequence.pad_sequences(X_test_more,maxlen=maxlen, padding='post')
test=sequence.pad_sequences(test,maxlen=maxlen, padding='post')

In [ ]:
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
# from keras.optimizers import Adam
from keras.optimizer_v1 import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error

In [ ]:
embeddings_index = {}
with open("/kaggle/input/glove840b300dtxt/glove.840B.300d.txt",encoding='utf8') as f:
    for line in f:
        
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [ ]:
word_index = tok.word_index
#prepare embedding matrix
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

In [ ]:
sequence_input = Input(shape=(maxlen, ))
x = Embedding(max_features, embed_size, weights=[embedding_matrix],trainable = False)(sequence_input)
x = SpatialDropout1D(0.2)(x)
x = Bidirectional(GRU(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
x = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool]) 
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
# preds = Dense(1)(x)
preds = Dense(6, activation="sigmoid")(x)
model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-3),metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
batch_size = 128
epochs = 4
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.9, random_state=233)

In [ ]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC AUC Score - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [ ]:
early = EarlyStopping(monitor="val_acc", mode="max", patience=5)
ra_val = RocAucEvaluation(validation_data=(X_val, y_val), interval = 1)
callbacks_list = [ra_val, early]

In [ ]:
model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),callbacks = callbacks_list,verbose=1)
#Loading model weights
# model.load_weights(filepath)
print('Predicting....')


In [ ]:
less_score = model.predict(X_test_less,batch_size=1024,verbose=1)
more_score = model.predict(X_test_more,batch_size=1024,verbose=1)
test_out = model.predict(test,batch_size=1024,verbose=1)

In [ ]:
valid_less = valid[['less_toxic']]
valid_less[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = less_score

cat_mtpl = {'obscene': 0.16, 'toxic': 0.32, 'threat': 1.5, 
            'insult': 0.64, 'severe_toxic': 1.5, 'identity_hate': 1.5}

for category in cat_mtpl:
    valid_less[category] = valid_less[category] * cat_mtpl[category]

valid_less['score'] = valid_less.loc[:, 'toxic':'identity_hate'].sum(axis=1)



In [ ]:
valid_more = valid[['more_toxic']]
valid_more[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = more_score
for category in cat_mtpl:
    valid_more[category] = valid_more[category] * cat_mtpl[category]

valid_more['score'] = valid_more.loc[:, 'toxic':'identity_hate'].sum(axis=1)

In [ ]:
(valid_less.score < valid_more.score).mean()

In [ ]:
test_score[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = test_out
for category in cat_mtpl:
    test_score[category] = test_score[category] * cat_mtpl[category]

sample['score'] = test_score.loc[:, 'toxic':'identity_hate'].sum(axis=1)

In [ ]:
sample.to_csv("submission.csv", index = False)